In [1]:
import time
import pandas as pd
import numpy as np
import datetime as dt
from sklearn import preprocessing
import multiprocessing as mp
import warnings
warnings.filterwarnings('ignore')

mp.cpu_count()

36

In [2]:
from fencoding_CPUs import (save_to_csv,
                               reduce_mem_usage,
                               EmptyElim,
                               FEncoding,
                               OutlDetect,
                               FImputation
                               )

In [3]:
from pathlib import Path
# home-credit-default-risk tables
if not Path("data/application_train.csv").is_file():
    ! wget https://www.dropbox.com/s/j9xwcj9ixki5t2l/home-credit-default-risk.zip?dl=0 -O data.zip
    ! unzip -q data.zip
    ! rm data.zip
# default-of-credit-card-clients-dataset
if not Path("data/default_ucr.csv").is_file():
    ! wget https://www.dropbox.com/s/lj0d7qez18ea7dx/UCI_Credit_Card.csv?dl=0 -O default_ucr.csv

In [4]:
# Read in the source datasets
dict = {
    'datasets':[
                # home-credit-default-risk tables
                pd.read_csv('./data/application_train.csv'),
                # default-of-credit-card-clients-datasets
                pd.read_csv('./data/default_ucr.csv'),
    ],
    'name_dropped_columns':
                [
                 # home-credit-default-risk tables
                 ['SK_ID_CURR', 'TARGET'],
                 # default-of-credit-card-clients-datasets
                 ['ID', 'default.payment.next.month']              
    ],   
}

In [5]:
X = dict['datasets'][1]
X = reduce_mem_usage(X)


Memory usage of dataframe is 5.72 MB
Memory usage after optimization is: 1.86 MB
Memory usage decreased by 67.5%


In [6]:
emptyelim = EmptyElim(n_jobs= -1, chunks = None)

In [7]:
X = emptyelim.fit_transform(X, file_path=None)#'test_out.csv')


 columns to drop: {}


In [8]:
fencoding = FEncoding(n_jobs = -1, chunks = None, rest_col_names=['ID', 'default.payment.next.month'])

In [9]:
fencoding.initialize_types(X, 
                           n_unique_val_th=50,
                           categor_columns_keep=[],
                           numer_columns_keep=['SEX', 'EDUCATION'],
                           return_dtype=True, 
                           file_name='out_dict.pkl')

{'categor_columns': ['LIMIT_BAL',
  'PAY_6',
  'BILL_AMT2',
  'PAY_4',
  'BILL_AMT6',
  'PAY_5',
  'PAY_2',
  'PAY_0',
  'PAY_AMT6',
  'BILL_AMT3',
  'BILL_AMT4',
  'BILL_AMT5',
  'PAY_AMT2',
  'PAY_AMT4',
  'PAY_AMT3',
  'PAY_AMT5',
  'BILL_AMT1',
  'PAY_AMT1',
  'MARRIAGE',
  'PAY_3'],
 'numer_columns': ['EDUCATION', 'AGE', 'SEX'],
 'time_columns': [],
 'categor_columns_dtypes': ['float32',
  'int8',
  'float32',
  'int8',
  'float32',
  'int8',
  'int8',
  'int8',
  'float32',
  'float32',
  'float32',
  'float32',
  'float32',
  'float32',
  'float32',
  'float32',
  'float32',
  'float32',
  'int8',
  'int8'],
 'numer_columns_dtypes': ['int8', 'int8', 'int8'],
 'time_columns_dtypes': []}

In [10]:
X = fencoding.encode_categor(X, method = 'OrdinalEncoder', file_path=None)

In [11]:
#X = fencoding.encode_categor(X, method = 'OneHotEncoder', file_path=None)

In [12]:
outldetect = OutlDetect(outliers_detection_technique = 'iqr_proximity_rule', n_jobs = -1, 
                                                                             chunks = None)

In [13]:
X = outldetect.fit_transform(X, file_path=None)


 col_outl_info (upper, lower) bounds: {'BILL_AMT3': (-14262.0, 33716.0), 'BILL_AMT4': (-14479.0, 33275.0), 'AGE': (8.5, 60.5), 'PAY_AMT1': (-5455.5, 10876.5), 'BILL_AMT5': (-14521.625, 32543.375), 'PAY_AMT2': (-5847.125, 11149.875), 'LIMIT_BAL': (-43.0, 101.0), 'BILL_AMT1': (-13627.5, 34514.5), 'BILL_AMT2': (-14033.5, 34076.5), 'EDUCATION': (-0.5, 3.5), 'PAY_AMT4': (-6570.875, 10962.125), 'PAY_AMT3': (-6643.625, 11165.375), 'PAY_AMT5': (-6365.0, 10619.0), 'SEX': (-0.5, 3.5), 'BILL_AMT6': (-14995.125, 32185.875), 'PAY_AMT6': (-6398.0, 10674.0)}


In [14]:

# For regression
fimputation_rb = FImputation('regression-based', fill_with_value = None, n_jobs = -1, chunks = None)
# For boosting
fimputation_tb = FImputation('tree-based', fill_with_value = 'extreme_values', n_jobs = -1, chunks = None)

In [15]:
XX = fimputation_rb.impute(X,  file_path=None)
XX = fimputation_tb.impute(X,  file_path=None)

In [16]:
XX

,EDUCATION,AGE,SEX,PAY_6,LIMIT_BAL,BILL_AMT2,PAY_4,BILL_AMT6,PAY_5,PAY_2,...,BILL_AMT4,BILL_AMT5,PAY_AMT2,PAY_AMT4,PAY_AMT3,PAY_AMT5,BILL_AMT1,PAY_AMT1,MARRIAGE,PAY_3
0,2.0,24.0,2.0,1,5,1563,0,14095,1,5,...,14627,14167,6302,0,0,0,3719,0,1,0
1,2.0,26.0,2.0,3,34,17405,3,1490,2,5,...,2039,2192,3334,3334,3334,0,36,0,2,3
2,2.0,34.0,2.0,2,79,15105,3,14199,2,3,...,14460,14288,531,3334,3334,3334,1029,551,2,3
3,2.0,37.0,2.0,2,42,5748,3,511,2,3,...,356,455,1708,2624,6057,1802,5587,1575,1,3
4,2.0,57.0,1.0,2,42,7736,3,16544,2,3,...,19012,17096,3899,6245,4445,5308,12522,1575,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,3.0,39.0,1.0,2,8,18830,3,14463,2,3,...,11407,1461,1629,2504,4653,4099,19055,7151,1,3
29996,3.0,43.0,1.0,2,67,18139,0,14095,2,0,...,11621,5965,3743,6698,6855,0,17608,1264,2,0
29997,2.0,37.0,1.0,2,17,2221,0,16740,2,6,...,18985,18220,0,3567,1773,1067,2881,0,2,5
29998,3.0,41.0,1.0,0,76,11054,3,4960,2,0,...,6550,9302,3623,1020,5368,4366,19391,7180,1,3
